# now that we have subcortical niftis for all subjects, let's create some masked versions

In [28]:
from __future__ import division
import os
import nibabel as nb
import numpy as np
out_dir = '/home/shared/2018/visual/hcp_cerebellum/'
import h5py

import matplotlib.pyplot as pl
%matplotlib inline

In [29]:

dims = {
    'ang':0,
    'ecc':1,
    'gain':2,
    'meanvol':3,
    'r2':4,
    'rfsize':5
}


In [32]:
def gmm_threshold(data,n_components=2,maxrange=100):

    from sklearn.mixture import GMM

    # fit gaussian mixture model to define r2 threshold
    gmm = GMM(n_components = n_components)
    gmm = gmm.fit(np.expand_dims(data,1))

    x = np.linspace(0,maxrange,10000)
    logprob, responsibilities = gmm.score_samples(np.expand_dims(x,1))

    pdf = np.exp(logprob)
    pdf_individual = responsibilities * pdf[:, np.newaxis]

    order = np.argsort([np.argmax(p) for p in pdf_individual.T])

    # find border between first and second gauss
    thresh = x[np.where(pdf_individual[:,order[1]]>pdf_individual[:,order[0]])[0]][0]

    return thresh

In [33]:
def determine_best_subjects(method,n=10):
    
    if method == 'glasser_v123':
        
        '''
        this method looks at explained variance in v123 glasser atlas to determine best subjects
    
        '''
        
        # determine best subjects based on v123
        with h5py.File(os.path.join(out_dir,'atlas.mat'), 'r') as mat:
            v1mask = np.ravel(mat['wang2015'].value==2)
            for roi in [3,4,5,6,7]:
                v1mask += np.ravel(mat['wang2015'].value==roi)

        all_mean_r2 = []
        for sj in range(181):      
            all_mean_r2.append(np.nanmedian(allresults[0,sj,4,v1mask]))

        best_subjects = np.argsort(all_mean_r2)[::-1][:n]

        print 'best subjects based on V123 pRF explained variance: %s'%best_subjects
        
    elif method == 'cerebellum':
        
        '''
        this method looks at explained variance in cerebellum to determine best subjects
    
        '''
        
        # load cerebellum_mask from the cerebellum atlas
        fncer = '/home/vanes/bin/fsl/data/atlases/Cerebellum/Cerebellum-MNIfnirt-maxprob-thr50-2mm.nii.gz'
        cerimg = nb.load(fncer)
        cdata = cerimg.get_data()
        cmask = (cdata!=0)

        # save cmask to check
        cmaskimg=nb.Nifti1Image(cmask,affine=cerimg.affine,header=cerimg.header)
        nb.save(cmaskimg,os.path.join(out_dir,'cmask.nii'))

        all_mean_r2 = []
        for sj in range(181):      

            # load the prf results nifti
            fn = os.path.join(out_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii'%sj)
            img = nb.load(fn)
            data = img.get_data()

            # mask r2s with cerebellum mask:
            r2s = np.ravel(data[cmask.astype(bool),4])

            all_mean_r2.append(np.nanmedian(r2s))

        best_subjects = np.argsort(all_mean_r2)[::-1][:n]

        print 'best subjects based on cerebellum pRF explained variance: %s'%best_subjects     
        
    elif method == 'cerebellum_gmm':
        
        '''
        this method looks at explained variance in cerebellum to determine best subjects
        In addition to the 'cerebellum' method, this method also separates 'signal' and 'noise' 
        voxel pools by fitting a gaussian mixture model (GMM) to the r2s and only regarding the signal pool
    
        '''
        
        # load cerebellum_mask from the cerebellum atlas
        fncer = '/home/vanes/bin/fsl/data/atlases/Cerebellum/Cerebellum-MNIfnirt-maxprob-thr50-2mm.nii.gz'
        cerimg = nb.load(fncer)
        cdata = cerimg.get_data()
        cmask = (cdata!=0)

        # save cmask to check
        cmaskimg=nb.Nifti1Image(cmask,affine=cerimg.affine,header=cerimg.header)
        nb.save(cmaskimg,os.path.join(out_dir,'cmask.nii'))

        all_mean_r2 = []
        for sj in range(181):      

            # load the prf results nifti
            fn = os.path.join(out_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii'%sj)
            img = nb.load(fn)
            data = img.get_data()

            # mask r2s with cerebellum mask:
            r2s = np.ravel(data[cmask,dims['r2']])

            thresh = gmm_threshold(r2s,2,100)

            # now combine cerebellum mask with r2 mask
            r2mask = (data[:,:,:,dims['r2']]>thresh)
            mask = r2mask*cmask

            # mask r2s with cerebellum mask:
            r2s = np.ravel(data[mask.astype(bool),dims['r2']])

            all_mean_r2.append(np.nanmedian(r2s))

        best_subjects = np.argsort(all_mean_r2)[::-1][:n]

        print 'best subjects based on signal voxels in cerebellum %s'%best_subjects  
    
    elif method == 'cerebellum_rois':
        '''
        This method looks at the median r2 within the cerebellum ROI determined on data from the average subject.
        '''
  
        
        roi_fn = os.path.join(out_dir,'masks','cer_retmaps','all_masks_together.nii')
        maskimg = nb.load(roi_fn)
        roimask = (maskimg.get_data()>0)
        
        all_r2 = []
        for sj in range(181):      

            # load the prf results nifti
            fn = os.path.join(out_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii'%sj)
            img = nb.load(fn)
            data = img.get_data()

            # mask r2s with roi mask:
            r2s = np.ravel(data[roimask,dims['r2']])    
            all_r2.append(np.nanmedian(r2s))

        best_subjects = np.argsort(all_r2)[::-1][:n]

        print 'best subjects based on median pRF explained variance within cerebellum ROI %s'%best_subjects


    return best_subjects





# mask voxels in different ways

In [49]:
def mask_voxels(data,mask_type,sj=None):
    """
    This function returns a mask bool where 
    1 = valid voxel
    0 = invalid voxel
    """
    
    if mask_type == 'none':
        """
        Simply a none mask
        """
        
        mask = np.ones_like(data).astype(bool)

    elif mask_type == 'r2':
        """
        this is the same mask as in the paper, 
        I've added the R2 for the weighted avg subject 
        by fitting a GMM to its r2s
        """
        
        # determine r2 threshold
        if sj in [181,182,183]:
            r2thresh=9.8
        elif sj == 184:
            r2thresh = 3.692
        else:
            r2thresh=2.2
        
        mask = (data[:,:,:,dims['r2']]>r2thresh)

    elif mask_type == 'r2_ecc':
        """
        This mask assumes there's two pools of voxels.
        The first is a pool of voxels that like the fovea
        and have no general interest for anything in the periphery.
        The logic here is that these voxels activate for 
        the increased difficulty when the bar passed fixation.
        """        
 
        # mask r2s with cerebellum mask:
        if sj in [181,182,183]:
            r2thresh=9.8
        elif sj == 184:
            r2thresh = 3.692            
        else:
            r2thresh=2.2
            
        # mask voxels with ecc < 0.1 degrees of visual angle
        eccthresh=0.1   
    
        # define masks
        r2mask = (data[:,:,:,dims['r2']]>r2thresh) # valid voxels
        eccmask = (data[:,:,:,dims['ecc']]>eccthresh) # valid voxels

        # now combine r2 and ecc mask with the spillovermask
        mask = r2mask*eccmask
        
#     elif mask_type == 'r2_eccgmm':
#         """
#         This mask assumes there's two pools of voxels.
#         The first is a pool of voxels that like the fovea
#         and have no general interest for anything in the periphery.
#         The logic here is that these voxels activate for 
#         the increased difficulty when the bar passed fixation.
#         """        
 
#         # mask r2s with cerebellum mask:
#         if sj in [181,182,183]:
#             r2thresh=9.8
#         elif sj == 184:
#             r2thresh = 3.692            
#         else:
#             r2thresh=2.2

#         eccthresh = gmm_threshold(np.ravel(data[:,:,:,dims['ecc']]),n_components=2,maxrange=12)
#         print eccthresh
    
#         # define masks
#         r2mask = (data[:,:,:,dims['r2']]>r2thresh) # valid voxels
#         eccmask = (data[:,:,:,dims['ecc']]>eccthresh) # valid voxels

#         # now combine r2 and ecc mask with the spillovermask
#         mask = r2mask*eccmask
        
    elif mask_type == 'r2_spill':
        """
        This applies the spill mask drawn on data from the avg subject.
        The 'spill' refers to activity from cortex that is averaged
        into the cerebellum by the smoothing and nonlinear transformations
        to MNI space.
        """                
        
        # determine r2 threshold
        if sj in [181,182,183]:
            r2thresh=9.8
        elif sj == 184:
            r2thresh = 3.692
        else:
            r2thresh=2.2

        r2mask = (data[:,:,:,dims['r2']]>r2thresh) 
        
        # load the spillover mask results nifti
        fn = os.path.join(out_dir,'masks','spillovermask.nii.gz')
        img = nb.load(fn)
        spillmask = img.get_data()
 
        # now combine r2 and ecc mask with the spillovermask
        mask = r2mask*(spillmask==0) # spillovermask is 0 for valid voxels
        
    elif mask_type == 'r2_spill_ecc':
        """
        This mask combines the spill and ecc masks
        """
        
        # load the spillover mask results nifti
        fn = os.path.join(out_dir,'masks','spillovermask.nii.gz')
        img = nb.load(fn)
        spillmask = img.get_data()
 
        # mask r2s with cerebellum mask:
        if sj in [181,182,183]:
            r2thresh=9.8
        elif sj == 184:
            r2thresh = 3.692            
        else:
            r2thresh=2.2

        # mask voxels with ecc < 0.5
        eccthresh=0.1   
    
        # define masks
        r2mask = (data[:,:,:,dims['r2']]>r2thresh) # valid voxels
        eccmask = (data[:,:,:,dims['ecc']]>eccthresh) # valid voxels

        # now combine r2 and ecc mask with the spillovermask
        mask = r2mask*eccmask*(spillmask==0) # spillovermask is 0 for valid voxels
        
#     elif mask_type == 'r2_spill_ecc':
#         """
#         This mask combines the spill and GMM ecc masks
#         """
        
#         # load the spillover mask results nifti
#         fn = os.path.join(out_dir,'masks','spillovermask.nii.gz')
#         img = nb.load(fn)
#         spillmask = img.get_data()
 
#         # mask r2s with cerebellum mask:
#         if sj in [181,182,183]:
#             r2thresh=9.8
#         elif sj == 184:
#             r2thrsh = 12.971            
#         else:
#             r2thresh=2.2

#         # determine threshold using GMM
#         eccthresh = gmm_threshold(np.ravel(data[:,:,:,dims['ecc']]),n_components=2,maxrange=12)

#         # define masks
#         r2mask = (data[:,:,:,dims['r2']]>r2thresh) # valid voxels
#         eccmask = (data[:,:,:,dims['ecc']]>eccthresh) # valid voxels

#         # now combine r2 and ecc mask with the spillovermask
#         mask = r2mask*eccmask*(spillmask==0) # spillovermask is 0 for valid voxels
        
    elif mask_type == 'r2_spill_size':
        
        # load the spillover mask results nifti
        fn = os.path.join(out_dir,'masks','spillovermask.nii.gz')
        img = nb.load(fn)
        spillmask = img.get_data()
 
        # mask r2s with cerebellum mask:
        if sj in [181,182,183]:
            r2thresh=9.8
        elif sj == 184:
            r2thresh = 3.692
        else:
            r2thresh=2.2

        # mask voxels with size < 0.15
        sizethresh=0.15
    
        # define masks
        r2mask = (data[:,:,:,dims['r2']]>r2thresh) # valid voxels
        sizemask = (data[:,:,:,dims['rfsize']]>sizethresh) # valid voxels

        # now combine r2 and ecc mask with the spillovermask
        mask = r2mask*sizemask*(spillmask==0) # spillovermask is 0 for valid voxels
        
    elif mask_type == 'r2_spill_fix':
        
        # load the spillover mask results nifti
        fn = os.path.join(out_dir,'masks','spillovermask.nii.gz')
        img = nb.load(fn)
        spillmask = img.get_data()
 
        # mask r2s with cerebellum mask:
        if sj in [181,182,183]:
            r2thresh=9.8
        elif sj == 184:
            r2thresh = 3.692
        else:
            r2thresh=2.2

        # mask voxels with size < 0.15
        # let's assume that voxels with ecc < 0.15 (i.e. fixation size)
        # AND size of < 0.15 are voxels that respond to fixation dot.
        sizethresh=0.15
        eccthresh = 0.15
        sizemask = (data[:,:,:,dims['rfsize']]<sizethresh) # invalid voxels
        eccmask = (data[:,:,:,dims['ecc']]<eccthresh) # invalid voxels
        fixmask = np.invert(sizemask*eccmask) # valid voxels

        # define masks
        r2mask = (data[:,:,:,dims['r2']]>r2thresh) # valid voxels

        # now combine r2 and ecc mask with the spillovermask
        mask = r2mask*fixmask*(spillmask==0) # spillovermask is 0 for valid voxels
                
    elif mask_type == 'r2_roi':
        
        roi_fn = os.path.join(out_dir,'masks','cer_retmaps','all_masks_together.nii')
        maskimg = nb.load(roi_fn)
        roimask = (maskimg.get_data()>0)        
        
        # create r2 mask
        if sj in [181,182,183]:
            r2thresh=9.8
        elif sj == 184:
            r2thresh = 3.692
        else:
            r2thresh=2.2
        r2mask = (data[:,:,:,dims['r2']]>r2thresh) # valid voxels

        mask = r2mask*roimask
        
#     elif mask_type == 'r2_buck':
        
#         # load cerebellum_mask from the cerebellum atlas to get the shape info
#         fn = os.path.join(out_dir,'buckner_dorsal_att.nii.gz')
#         img = nb.load(fn)
#         roimask = img.get_data().astype(bool)
        
#         # create r2 mask
#         if sj in [181,182,183]:
#             r2thresh=9.8
        
#         elif sj == 184:
#             r2thrsh = 3.692
#         else:
#             r2thresh=2.2
#         r2mask = (data[:,:,:,dims['r2']]>r2thresh) # valid voxels

#         mask = r2mask*roimask        
 
#     elif mask_type == 'gmmr2_roi':
        
#         # load cerebellum_mask from the cerebellum atlas to get the shape info
#         fncer = '/home/vanes/bin/fsl/data/atlases/Cerebellum/Cerebellum-MNIfnirt-maxprob-thr50-2mm.nii.gz'
#         cerimg = nb.load(fncer)
#         roimask = np.zeros(cerimg.shape)
        
#         # now create roi mask out of selected rois:
#         rois = ['left_ventral','right_ventral']#,'left_central','right_central']
#         for roi in rois:
#             roi_fn = os.path.join(out_dir,roi+'.nii.gz')
#             roiimg = nb.load(roi_fn)
#             roidata = roiimg.get_data() 
#             roimask[roidata==1] = 1
#         roimask = (roimask == 1) # these are the valid voxels
        
#         # use r2s from whole brain
#         r2s = np.ravel(allresults[0,sj,dims['r2'],:])
#         r2thresh = gmm_threshold(r2s)
# #         r2thresh = gmm_threshold(np.ravel(data[:,:,:,dims['r2']]))
#         print r2thresh
#         r2mask = (data[:,:,:,dims['r2']]>r2thresh) # valid voxels

#         mask = r2mask*roimask
        
    return mask


### now that we have the best subjects, let's create a 'best_subjects' folder with differently masked niftis in there

Let's do masking as follows:

1. as in paper: r_squared for the avg subjects at 9.8, for the individual subjects at 2.2
2. same r2 mask as in paper, but also include ecc mask > 0.5 dva

In [52]:
best_subjects = determine_best_subjects(method = 'cerebellum_rois',n=181)
order = [np.where(best_subjects==s)[0][0] for s in range(181)]

print order

for sj in np.hstack([[183,184],best_subjects]):#np.hstack([range(181),183]):
    
    print('now creating masked niftis for subject %d'%sj)

    if sj == 183:
        mask_types =  ['r2','r2_spill','r2_spill_fix','r2_roi']#,'r2_spill_size','r2_roi']#'r2_ecc']#['r2','r2_spill','r2_spill_ecc','r2high_spill','r2_buck']
        rank = 'avg'
    elif sj == 184:
        rank = 'wavg'
        mask_types = ['r2','r2_spill','r2_spill_fix','r2_roi']
    else:
        mask_types = ['r2_roi','r2_spill_fix']#['r2_roi']
        rank = str(np.where(best_subjects==sj)[0][0])
        
    for mask_type in mask_types:
    
        # try to create mask dir
        mask_dir = os.path.join(out_dir,'masked_niftis',mask_type)
        if not os.path.isdir(mask_dir): os.mkdir(mask_dir)

        # load the prf results nifti
        fn = os.path.join(out_dir,'all_subjects','prfresults_subject_%s.dscalar_data_sub.nii'%sj)        
        img = nb.load(fn)
        data = img.get_data()

        # determine the mask
        mask = mask_voxels(data,mask_type,sj)
        
        #mask data
        data[~mask] = np.nan

        # save datae
        out_fn = os.path.join(mask_dir,'prfresults_subject_rank_%s.nii'%rank)        
        new_data = nb.Nifti1Image(data,affine=img.affine,header=img.header)
        nb.save(new_data,out_fn[:-4])

        # save dimensions separately (for surface plots)
        for m in ['ang','ecc','rfsize','r2']:#,'dist','instim']:
            out_fn = os.path.join(mask_dir,'prfresults_subject_rank_%s_%s.nii'%(rank,m))        
            new_data = nb.Nifti1Image(data[:,:,:,dims[m]],affine=img.affine,header=img.header)
            nb.save(new_data,out_fn[:-4])    



best subjects based on median pRF explained variance within cerebellum ROI [  7 159 169  45  69  25 141 149   3 135  50  70 156  27 101  48 129  47
  63  58  98 167 116 134 162  20  56  62 150  76 170 171 147  46   8  34
 109  72 113  94 122 151  80 125  77  37  12 146 148 153  85  10 138  38
  26   5  84   9  67 179  43  39  55  36 120 158 112 137 108 139 161 140
 100  14  90  35  87  28 168  91   4 111  32  16 131 177 145 127  97  42
  23 110  17  74  41  89  75 176 105 124  99   2  44  24 136  83 118  93
  78  96 128 175  73  19 142 123 117  18  13 173  59  88 121  71  51  31
 178 166  81  33 126 133 164 155 119  64  66  82  21  92  49  15 172  79
 114   6 102  60  29 115 160  54 132 157  22  40  95  30  11  65 180 144
  61 165 130  53 143  52 163  57  68  86   1 107 154 152 106 103   0 104
 174]
[178, 172, 101, 8, 80, 55, 145, 0, 34, 57, 51, 158, 46, 118, 73, 141, 83, 92, 117, 113, 25, 138, 154, 90, 103, 5, 54, 13, 77, 148, 157, 125, 82, 129, 35, 75, 63, 45, 53, 61, 155, 94, 89, 60